In [1]:
import mlflow
# Connect to Databricks workspace with submitted credentials or use stored credentials
mlflow.login()
# Set tracking URI to Databricks -  tell MLflow to send the data into Databricks Workspace
mlflow.set_tracking_uri("databricks")

2025/10/29 02:46:22 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://dbc-7d1169bb-4536.cloud.databricks.com.


In [2]:
import os;
import mlflow
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

experiment_name = f"/Users/{os.environ['USER_EMAIL']}/agen-no-memory-{datetime.now()}"
try:
    mlflow.create_experiment(experiment_name)
except Exception as e:
    print(f"Experiment {experiment_name} already exists.")
mlflow.set_experiment(experiment_name)


with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        python_model="agent.py",  # replace with your relative path to agent code
        name="agent",
    )

2025/10/29 02:46:36 INFO mlflow.pyfunc: Predicting on input example to validate output


🏃 View run bright-lynx-64 at: https://dbc-7d1169bb-4536.cloud.databricks.com/ml/experiments/655080271598964/runs/530dfc5e649f4c23a1a9665ca84d578d
🧪 View experiment at: https://dbc-7d1169bb-4536.cloud.databricks.com/ml/experiments/655080271598964


AttributeError: 'dict' object has no attribute 'type'